# Goal:
I've got a phone interview for a job that I'd really like to get so I'm going to scrape some of the books and hopefully get a better general idea of the stuff from the company.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Scraping Libraries
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
#from selenium.webdriver.chrome.options import Options
from time import sleep
import re

In [3]:
def open_browser(alt_user_name = 'Thank you for your website', path = '../../Garage/chromedriver'):
    opts = webdriver.chrome.options.Options()
    opts.add_argument("user-agent=" + str(alt_user_name))
             # Path to Chromedriver
    return webdriver.Chrome(executable_path = path)#, options=opts)

In [99]:
book_list# = []

In [109]:
len(book_list)

330

In [110]:
browser = open_browser()

In [5]:
browser.get('https://www.amazon.com/s?k=rockridge+press&lo=grid&qid=1553032528&ref=sr_pg_1')

In [111]:
browser.get('https://www.amazon.com/s?k=althea+press')

In [97]:
len([x for x in soup_list if x != []])

48

Doing this by hand to be safe and avoid angering the Amazon Gods

In [118]:
browser.find_element_by_class_name('a-last').click()            # Clicks the next page

soup = BeautifulSoup(browser.page_source)                       # Takes page and turns it into soup
soup = soup.find(attrs = {'class':'s-result-list sg-row'})      # Cuts down to only the stuff we care about
soup_list = []                                                  # Inits empty list 
for num in range(100):
    soup_list.append(soup.find_all(attrs = {'data-index':str(num)})) # Finds each item found and attaches it to list
soup_list = [x for x in soup_list if x != []]                   # Removing any items over the total number of results
for item in soup_list:                                          # Takes each item and adds it to the list of lists
    book_list.append([x.strip(' ') for x in item[0].text.split('\n') if x.strip(' ') != ''])
print(len(book_list))

646


In [120]:
# Saving what I have at this current point just to be safe.
pd.DataFrame(book_list).to_csv('../data/raw_book_data.csv', index = False)

In [188]:
book_processing = [{'list_form' : book} for book in book_list]

In [189]:
# Best Seller
for book in book_processing:
    if book['list_form'][0] == 'Best Seller':
        book['Best_seller'] =  book['list_form'][1]
    else:
        book['Best_seller'] = False

In [190]:
# Author
for book in book_processing:
    book['author'] = None
    for list_num in range(len(book['list_form'])):
        if book['list_form'][list_num] == 'by':
            book['author'] = book['list_form'][list_num + 1]
            if book['list_form'][list_num + 2] == 'and':
                book['co_author'] = book['list_form'][list_num + 3]
        else:
            pass

In [191]:
# Title
for book in book_processing:
    if book['list_form'][0] == 'Best Seller':
        book['title'] = book['list_form'][2]
    else:
        book['title'] = book['list_form'][0]

In [235]:
# Star Rating
for book in book_processing:
    for list_num in range(len(book['list_form'])):
        if book['list_form'][list_num][-5:] == 'stars':
            book['star_rating'] = book['list_form'][list_num][0:3]
            book['num_ratings'] = book['list_form'][list_num + 1]
        else:
            pass
    #book['title'] = book['list_form'][0]

In [236]:
# Resell Data
for book in book_processing:
    book['more_buying_choices'] = None
    book['Num_of_offers'] = None
    for list_num in range(len(book['list_form'])):
        if book['list_form'][list_num][0:19] == 'More Buying Choices':
            book['more_buying_choices'] = book['list_form'][list_num].strip('More Buying Choices')
            book['Num_of_offers'] = book['list_form'][list_num + 1].strip('(').strip(' used & new offers)')
        else:
            pass
    #book['title'] = book['list_form'][0]

In [237]:
# Formats
for book in book_processing:
    book['formats'] = []
    for list_num in range(len(book['list_form'])):
        if   book['list_form'][list_num] == 'Paperback':
            book['formats'].append('Paperback')
            
        elif book['list_form'][list_num] == 'Kindle':
            book['formats'].append('Kindle')
            
        elif book['list_form'][list_num] == 'Spiral-bound':
            book['formats'].append('Spiral-bound')
            
        elif book['list_form'][list_num] == 'Hardcover':
            book['formats'].append('Hardcover')
        else:
            pass
    #book['title'] = book['list_form'][0]

In [238]:
# List of prices
for book in book_processing:
    book['prices'] = []
    for list_num in range(len(book['list_form'])):
        for symbol in book['list_form'][list_num]:
            if symbol == '$' and len(book['list_form'][list_num]) <= 15:
                book['prices'].append(book['list_form'][list_num])

In [211]:
book_processing[0:5]

[{'list_form': ['Best Seller',
   'in Cancer Cookbooks',
   'Clean Keto Lifestyle: The Complete Guide to Transforming Your Life and Health',
   'by',
   'Karissa Long',
   '4.6 out of 5 stars',
   '34',
   'Paperback',
   '$18.01$18.01',
   '$25.99$25.99',
   'Get it as soon as Tomorrow, Mar 20',
   'FREE Shipping on orders over $25 shipped by Amazon',
   'More Buying Choices$16.82',
   '(25 used & new offers)',
   'Other format:',
   'Kindle'],
  'Best_seller': 'in Cancer Cookbooks',
  'author': 'Karissa Long',
  'title': 'Clean Keto Lifestyle: The Complete Guide to Transforming Your Life and Health',
  'star_rating': '4.6 out of 5 stars',
  'num_ratings': '34',
  'more_buying_choices': '$16.82',
  'Num_of_offers': '25',
  'formats': ['Paperback', 'Kindle'],
  'prices': ['$18.01$18.01', '$18.01$18.01', '$25.99$25.99', '$25.99$25.99']},
 {'list_form': ['The Diabetic Cookbook for Electric Pressure Cookers: Instant Healthy Meals for Managing Diabetes',
   'by',
   'Shelby Kinnaird',
   '

In [239]:
df = pd.DataFrame(book_processing, columns= [
    'title',
    'prices',
    'star_rating',
    'Best_seller',
    'author',
    'co_author',
    'formats',
    'list_form',
    'more_buying_choices',
    'Num_of_offers',
    'num_ratings'
])


In [241]:
df.to_csv('../data/organized_books.csv', index = False)

In [251]:
df['to_delete'] = df['formats'].map(lambda x: (len(x)))

In [ ]:
lambda x: x if (x<3)

In [242]:
mask = df['Best_seller'] == False
df[mask]
#df.drop()['formats'].value_counts()

,title,prices,star_rating,Best_seller,author,co_author,formats,list_form,more_buying_choices,Num_of_offers,num_ratings
1,The Diabetic Cookbook for Electric Pressure Co...,"[$6.99$6.99, $6.99$6.99]",4.2,False,Shelby Kinnaird,Simone Harounian MS RD CDN CDE,"[Kindle, Paperback]",[The Diabetic Cookbook for Electric Pressure C...,None,None,17
2,"The Skinnytaste Cookbook: Light on Calories, B...","[$22.13$22.13, $22.13$22.13, $30.00$30.00, $30...",4.7,False,Gina Homolka,NaN,"[Hardcover, Kindle, Paperback]","[The Skinnytaste Cookbook: Light on Calories, ...",$10.87,154,"2,355"
3,The Healthy Sheet Pan Cookbook: Satisfying One...,"[$10.49$10.49, $10.49$10.49, $13.99$13.99, $13...",4.0,False,Ruthy Kirwan,NaN,"[Paperback, Kindle]",[The Healthy Sheet Pan Cookbook: Satisfying On...,$5.44,17,34
5,Mediterranean Diet for Beginners: The Complete...,"[$8.49$8.49, $8.49$8.49, $9.99$9.99, $9.99$9.99]",3.9,False,Rockridge Press,NaN,"[Paperback, Kindle, Spiral-bound]",[Mediterranean Diet for Beginners: The Complet...,$4.98,88,785
7,The Anti Inflammatory Diet Cookbook: No Hassle...,"[$0.00$0.00, $0.00$0.00, Or $6.99 to buy]",4.2,False,Madeline Given NC,NaN,"[Kindle, Paperback]",[The Anti Inflammatory Diet Cookbook: No Hassl...,None,None,156
8,Paleo Cookbook: 300 Delicious Paleo Diet Recipes,"[$7.83$7.83, $7.83$7.83, $14.99$14.99, $14.99$...",4.2,False,Rockridge Press,NaN,"[Paperback, Kindle, Hardcover]",[Paleo Cookbook: 300 Delicious Paleo Diet Reci...,$1.54,128,463
9,"Healthy Cookbook for Two: 175 Simple, Deliciou...","[$6.99$6.99, $6.99$6.99]",4.3,False,Rockridge Press,NaN,"[Kindle, Paperback]","[Healthy Cookbook for Two: 175 Simple, Delicio...",None,None,286
10,Indian Instant Pot® Cookbook: Traditional Indi...,"[$0.00$0.00, $0.00$0.00, Or $6.99 to buy]",4.6,False,Urvashi Pitre,NaN,"[Kindle, Paperback]",[Indian Instant Pot® Cookbook: Traditional Ind...,None,None,782
12,Vegan Cookbook for Beginners: The Essential Ve...,"[$11.99$11.99, $11.99$11.99]",4.3,False,Rockridge Press,NaN,"[Paperback, Kindle]",[Vegan Cookbook for Beginners: The Essential V...,$2.48,77,352
14,The Ultimate Instant Pot® Cookbook for Two: Pe...,"[$13.59$13.59, $13.59$13.59, $16.99$16.99, $16...",4.1,False,Janet A. Zimmerman,NaN,"[Paperback, Kindle]",[The Ultimate Instant Pot® Cookbook for Two: P...,$9.32,17,29


In [233]:
mask = df['formats'] == 
type(df['formats'][0])#[mask]

list

In [89]:
try:
    browser.find_element_by_class_name('a-last').click()
except:
    print("Done!")

[<selenium.webdriver.remote.webelement.WebElement (session="c24e500bcbb1719b9d12f6db05bfdbf7", element="0.7186844993204746-1")>]

In [254]:
df.isna().sum()

title                    0
prices                   0
star_rating            189
Best_seller              0
author                 240
co_author              627
formats                  0
list_form                0
more_buying_choices    322
Num_of_offers          322
num_ratings            189
to_delete                0
dtype: int64

In [34]:
potential_item = items[0][0].find_all(attrs = {'class' : 'sg-col-inner'})

In [70]:
for item in items:
    #print(item[0])
    print([x.strip(' ') for x in item[0].text.split('\n') if x.strip(' ') != ''])

['Best Seller', 'in Cancer Cookbooks', 'Clean Keto Lifestyle: The Complete Guide to Transforming Your Life and Health', 'by', 'Karissa Long', '4.6 out of 5 stars', '34', 'Paperback', '$18.01$18.01', '$25.99$25.99', 'Get it as soon as Tomorrow, Mar 20', 'FREE Shipping on orders over $25 shipped by Amazon', 'More Buying Choices$16.82', '(25 used & new offers)', 'Other format:', 'Kindle']
['The Diabetic Cookbook for Electric Pressure Cookers: Instant Healthy Meals for Managing Diabetes', 'by', 'Shelby Kinnaird', 'and', 'Simone Harounian MS RD CDN CDE', '4.2 out of 5 stars', '17', 'Kindle', '$6.99$6.99', 'Other format:', 'Paperback']
['The Skinnytaste Cookbook: Light on Calories, Big on Flavor', 'by', 'Gina Homolka', '4.7 out of 5 stars', '2,355', 'Hardcover', '$22.13$22.13', '$30.00$30.00', 'Get it as soon as Thu, Mar 21', 'FREE Shipping on orders over $25 shipped by Amazon', 'More Buying Choices$10.87', '(154 used & new offers)', 'Other formats:', 'Kindle', ',', 'Paperback']
['The Health

In [78]:
test_item = potential_item[0]

[<div class="s-expand-height s-include-content-margin s-border-bottom">
 <div class="a-section a-spacing-medium">
 <div class="sg-row">
 <div class="sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32"><div class="sg-col-inner">
 <div class="a-section a-spacing-micro s-min-height-extra-large">
 <a class="a-link-normal" href="/gp/bestsellers/books/746214/ref=sr_bs_0_746214_1">
 <div class="a-row a-badge-region"><span aria-labelledby="1641523255-label 1641523255-supplementary" class="a-badge" data-a-badge-supplementary-position="right" data-a-badge-type="status" id="1641523255" tabindex="0"><span aria-hidden="true" class="a-badge-label" data-a-badge-color="sx-orange" id="1641523255-label"><span class="a-badge-label-inner a-text-ellipsis">
 <span class="a-badge-text" data-a-badge-color="sx-cloud">Best Seller</span>
 </span></span><span aria-hidden="true" class="a-badge-supplementary-text a-text-ellipsis" id="1641523255-supplement

In [52]:
print(potential_item[0].text)









Best Seller
in Cancer Cookbooks




















Clean Keto Lifestyle: The Complete Guide to Transforming Your Life and Health


by 

    
        
        
            Karissa Long
        
    







4.6 out of 5 stars




34









    
        
        
            Paperback
        
    



$18.01$18.01
$25.99$25.99













Get it as soon as Tomorrow, Mar 20



FREE Shipping on orders over $25 shipped by Amazon




More Buying Choices$16.82

    
        
        
            (25 used & new offers)
        
    




Other format: 

    
        
        
            Kindle
        
    














In [ ]:
book_dict = {
    'title':,
    'author':,
    'photo_link':,
    'rating':,
    'hardcover_y':,
    'hardcover_price':,
    'paperback_y':,
    'paperback_price':,
    'kindle_y':,
    'kindle_price':,
    
}